<a href="https://colab.research.google.com/github/greyarea01/MoEDAL-ML-ANA/blob/main/TrainCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#  mount google drive to get access to the data sets
from google.colab import drive
drive.mount('/content/drive')


# clone the repo with the modules (and this file)
!git clone https://github.com/greyarea01/MoEDAL-ML-ANA.git

# add work area to the PYTHONPATH
import sys
sys.path.append('/content/MoEDAL-ML-ANA/modules')

imagePath='/content/drive/My Drive/Colab Notebooks/moedal-examples/pits_frontback_2d'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'MoEDAL-ML-ANA' already exists and is not an empty directory.


In [5]:
import numpy as np
import pickle

import matplotlib.pyplot as plt
def open_pits(file):
    pits = 0
    with open(file,'rb') as pickle_file:
        pits = pickle.load(pickle_file)
    # pits is a dictionary containing signal pits and background pits,
    # for front surface, and reverse surface (r)

    sig = pits['sig']
    sigr = pits['sigr']
    bkg = pits['bkg']
    bkgr = pits['bkgr']
    # pits are stored as a list of tuples
    # pit_tuple = (slide index, x_pos, y_pos, 28*28 numpy array / image)

    pit = sigr[0] # take 1st element in list
    pit = pit[3] # take the image

    sig_images = [ z[3] for z in sig ] # convert sig into a list of images
    sig_array = np.asarray(sig_images) #can convert into array
    bkg_images = [ j[3] for j in bkg ]
    bkg_array = np.asarray(bkg_images)

    """
    making labeled datasets from these lists

    Either use ds label on a list to make a dataset
    this can be concatenated and shuffled with the bkg dataset

    Or generate a label array (see ds label), and use the arrays directly
    """

#    sig_dataset = ds_label(sig_images,(1.0,0.0))
#    bkg_dataset = ds_label(bkg_images,(0.0,1.0))
    sig_label = [(1.0,0.0) for l in sig_array]
    bkg_label = [(0.0,1.0) for l in bkg_array]

    return sig_array, bkg_array, sig_label, bkg_label

In [6]:
from tensorflow import keras
from tensorflow.keras import layers

def k_sequential():
    """
    Default model used in Xe Ion study
    """
    model = keras.Sequential()
    model.add(keras.Input(shape=(28,28,1))) # specifying input
    model.add(layers.Conv2D(32, kernel_size = (3,3), padding='same',activation='tanh'))
    model.add(layers.MaxPooling2D(padding='same'))
    model.add(layers.Conv2D(32, kernel_size = (3,3), padding='same',activation='tanh'))
    model.add(layers.MaxPooling2D(padding='same'))
    model.add(layers.Conv2D(32, kernel_size = (3,3), padding='same',activation='tanh'))
    model.add(layers.MaxPooling2D(padding='same'))
    model.add(layers.Conv2D(32, kernel_size = (3,3), padding='same',activation='tanh'))
    model.add(layers.MaxPooling2D(padding='same'))
    model.add(layers.GlobalMaxPooling2D()) # compress to 1 spatial dimension
    model.add(layers.Dense(32, activation = 'relu'))
    model.add(layers.Dense(2, activation = 'softmax'))
    model.summary()
    return model

In [11]:
from sklearn.model_selection import train_test_split
def training():
    model = k_sequential() # create the compiled model
    s,b,sl,bl = open_pits(imagePath+'/pits_checked') # get the data

    print('opened the inputs',flush=True)
    print(s.shape)
    print(b.shape)

    data_all = np.concatenate((s,b),axis=0)
    label_all = np.concatenate((sl,bl),axis=0)
    print(data_all.shape,label_all.shape)


    data_train, data_test, label_train, label_test  = train_test_split(data_all,label_all,test_size=0.5,random_state=42)
    print(len(data_train),len(label_train))
#
    print('hello', flush=True)


    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[keras.metrics.BinaryAccuracy, keras.metrics.BinaryCrossentropy],run_eagerly=False)
    print('model compiled',flush=True)
    history = model.fit(data_train,label_train, batch_size = 10, epochs=7,verbose=1,validation_data=(data_test,label_test))

    print('Done',flush=True)
    return history,model

In [12]:
training()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)                    │ (None, 28, 28, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 14, 14, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 7, 7, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 7, 7, 32)            │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 4, 4, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 4, 4, 32)            │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 2, 2, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling2d_2               │ (None, 32)                  │               0 │
│ (GlobalMaxPooling2D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 2)                   │              66 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 29,186 (114.01 KB)

 Trainable params: 29,186 (114.01 KB)

 Non-trainable params: 0 (0.00 B)

opened the inputs
(1299, 28, 28)
(1471, 28, 28)
(2770, 28, 28) (2770, 2)
1385 1385
hello
model compiled
Epoch 1/7
139/139 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - binary_accuracy: 0.7223 - binary_crossentropy: 0.5666 - loss: 0.5196 - val_binary_accuracy: 0.9242 - val_binary_crossentropy: 0.3259 - val_loss: 0.2468
Epoch 2/7
139/139 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - binary_accuracy: 0.9230 - binary_crossentropy: 0.3214 - loss: 0.2477 - val_binary_accuracy: 0.9264 - val_binary_crossentropy: 0.2683 - val_loss: 0.2333
Epoch 3/7
139/139 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - binary_accuracy: 0.9405 - binary_crossentropy: 0.2565 - loss: 0.1936 - val_binary_accuracy: 0.9271 - val_binary_crossentropy: 0.3051 - val_loss: 0.2305
Epoch 4/7
139/139 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - binary_accuracy: 0.9341 - binary_crossentropy: 0.2673 - loss: 0.2070 - val_binary_accuracy: 0.9141 - val_binary_crossentropy: 0.3143 - val_loss: 0.2533
Epoch 5/7
139/139 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - binary_accuracy: 0

(<keras.src.callbacks.history.History at 0x7ea064951810>,
 <Sequential name=sequential_2, built=True>)